In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pd
import json
import re

# Read recipe inputs
trans_json = dataiku.Dataset("Trans_json")
trans_json_df = trans_json.get_dataframe()
trans_json_df = trans_json_df.dropna(subset=['transcript_json.callData'])

In [ ]:
trans = trans_json_df['transcript_json.callData']

#new_df = pd.DataFrame(columns = ['NER','score','duree',"nom"])
liste_dic = []
liste_type_2 = []
liste_score = []
liste_score_2 = []
liste_duree = []
liste_duree_2 = []




for i in range(len(trans)):
    #dic_inter = {}
    id_conv = trans_json_df['metadata.reconciliation_id'][i]
    #dic_inter['nom'] = id_conv
    one_trans = trans[i]
    trans_dict = json.loads(one_trans)
    #liste_type_par_conversation = []
    #liste_score_par_conversation = []
    #liste_duree_par_conversation = []
    #liste_cat_par_conversation = []

    for l in trans_dict:
       
        
        word = l['words']
        for m in word:
            value = m['value']
            matchObj = re.findall('\[.+\]', value)
            
            if matchObj:
                    dic_inter = {}
                    #liste_type_par_conversation.append(matchObj)
                    dic_inter['nom'] = id_conv
                    dic_inter['NER'] = matchObj
                    tdp = l['content']
                    dic_inter['Contexte'] = tdp


                    start = float(m['start'])
                    if start < 0.02:
                        dic_inter['start'] = start
                    else:
                        dic_inter['start'] = start-0.02


                    duree = float(m['length'])
                    #liste_duree_par_conversation.append(duree)
                    dic_inter['duree'] = duree

                    end = duree+start+0.02
                    dic_inter['end'] = end

                    score = float(m['score']) 
                    #liste_score_par_conversation.append(score)
                    dic_inter['score'] = score

                    if score>= 0 and score < 0.7:
                        cat = 'PAS SATISFAISANT'
                        #liste_cat_par_conversation.append(cat)
                        dic_inter['categorie_satisfaction'] = cat

                    if score>= 0.7 and score < 0.8:
                        cat = 'SATISFAISANT'
                        #liste_cat_par_conversation.append(cat)
                        dic_inter['categorie_satisfaction'] = cat

                    if score>= 0.8:
                        cat = 'TRES SATISFAISANT'
                        #liste_cat_par_conversation.append(cat)
                        dic_inter['categorie_satisfaction'] = cat
                    
                    liste_dic.append(dic_inter)
                
               
                

                

    #dic_par_ligne[id_conv] = liste_type_par_conversation
#print(liste_dic)

In [ ]:
new_df = pd.DataFrame(liste_dic)
output_test_test = dataiku.Dataset("output_test_test")
output_test_test.write_with_schema(new_df)